In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='segnet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/e10d44333b3e4cf98a71083f1b56f5f7



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
#from Model import *
from Model_128 import *
home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_128_channels_30epoch', type=str, help="name of the log") #debug model_intersect
parser.add_argument("-gpu", default='1', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=30, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 30
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_128_channels_30epoch
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
(?, 32, 128, 1)
(?, 32, 128, 1)
(?, 32, 128, 128)
111111
(?, 16, 64, 128)
(?, 16, 64, 128)
22222
(?, 8, 32, 128)
(?, 8, 32, 128)
33333
(?, 4, 16, 128)
(?, 4, 16, 128)
44444
(?, 2, 8, 128)
(?, 2, 8, 128)
d4444444
(?, 4, 16, 128)
d333333
(?, 8, 32, 128)
d22222
(?, 16, 64, 128)
d111111
(?, 32, 128, 128)
cv
(?, 32, 128, 2)
(2,)
(?, 32, 128, 2)
loss: ()
INFO:tensorflow:Summary name conv_classifier/weight_loss (raw) is illegal; using conv_classifier/weight_loss__raw_ instead.
INFO:tensorflow:Summary name loss/cr

COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
Ran global_variables_initializer
Epoch: 1  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.10383167
accuracy = 0.537611
mean IU  = 0.290732
    class # 0 capture rate = 0.538356 
    class # 1 capture rate = 0.524418 
TRAIN: Batch: 0.003908998514580564 Loss: 0.065938644
accuracy = 0.612213
mean IU  = 0.350434
    class # 0 capture rate = 0.599637 
    class # 1 capture rate = 0.819932 
TRAIN: Batch: 0.007817997029161129 Loss: 0.09349815
accuracy = 0.597480
mean IU  = 0.341719
    class # 0 capture rate = 0.586901 
    class # 1 capture rate = 0.753657 
TRAIN: Batch: 0.011726995543741693 Loss: 0.0649424
accuracy = 0.763660
mean IU  = 0.414707
    class # 0 capture rate = 0.785200 
    class # 1 capture rate = 0.355263 
TRAIN: Batch: 0.015635994058322257 Loss: 0.06772768
accuracy = 0.775277
mean IU  = 0.453880
    class # 0 capture rate = 0.786093 
    class # 1 capture rate = 0.607961 
TRAIN: Batch: 0.019544992572902823 Loss: 0.07154444
accuracy = 0.721094
mean IU  = 0.387379
    class # 0 capture rate = 0.736798 
    class # 1 capture rate = 0.38683

...

TRAIN: Batch: 0.574622781643343 Loss: 0.009718582
accuracy = 0.963863
mean IU  = 0.814887
    class # 0 capture rate = 0.962616 
    class # 1 capture rate = 0.979351 
TRAIN: Batch: 0.5785317801579235 Loss: 0.0068147406
accuracy = 0.977098
mean IU  = 0.854540
    class # 0 capture rate = 0.976524 
    class # 1 capture rate = 0.985496 
TRAIN: Batch: 0.5824407786725041 Loss: 0.0043245773
accuracy = 0.985327
mean IU  = 0.889592
    class # 0 capture rate = 0.985208 
    class # 1 capture rate = 0.987272 
TRAIN: Batch: 0.5863497771870847 Loss: 0.0052004023
accuracy = 0.981027
mean IU  = 0.868246
    class # 0 capture rate = 0.980600 
    class # 1 capture rate = 0.987735 
TRAIN: Batch: 0.5902587757016652 Loss: 0.006662236
accuracy = 0.969545
mean IU  = 0.816142
    class # 0 capture rate = 0.968092 
    class # 1 capture rate = 0.991974 
TRAIN: Batch: 0.5941677742162458 Loss: 0.0063813333
accuracy = 0.974215
mean IU  = 0.816652
    class # 0 capture rate = 0.973354 
    class # 1 capture 

TRAIN: Batch: 0.7661637088577906 Loss: 0.0097916415
accuracy = 0.952327
mean IU  = 0.788517
    class # 0 capture rate = 0.948642 
    class # 1 capture rate = 0.993990 
TRAIN: Batch: 0.7700727073723712 Loss: 0.004882147
accuracy = 0.973288
mean IU  = 0.842979
    class # 0 capture rate = 0.971679 
    class # 1 capture rate = 0.995638 
TRAIN: Batch: 0.7739817058869518 Loss: 0.006474407
accuracy = 0.967232
mean IU  = 0.813179
    class # 0 capture rate = 0.965243 
    class # 1 capture rate = 0.996199 
TRAIN: Batch: 0.7778907044015323 Loss: 0.0065904274
accuracy = 0.973558
mean IU  = 0.839408
    class # 0 capture rate = 0.972339 
    class # 1 capture rate = 0.991278 
TRAIN: Batch: 0.7817997029161129 Loss: 0.006656675
accuracy = 0.977487
mean IU  = 0.833913
    class # 0 capture rate = 0.977623 
    class # 1 capture rate = 0.975000 
TRAIN: Batch: 0.7857087014306935 Loss: 0.009460167
accuracy = 0.962083
mean IU  = 0.769451
    class # 0 capture rate = 0.961087 
    class # 1 capture r

TRAIN: Batch: 0.9577046360722383 Loss: 0.005236416
accuracy = 0.981248
mean IU  = 0.868638
    class # 0 capture rate = 0.980592 
    class # 1 capture rate = 0.991718 
TRAIN: Batch: 0.9616136345868188 Loss: 0.008520597
accuracy = 0.964321
mean IU  = 0.785061
    class # 0 capture rate = 0.963053 
    class # 1 capture rate = 0.985640 
TRAIN: Batch: 0.9655226331013994 Loss: 0.0055585518
accuracy = 0.980222
mean IU  = 0.870879
    class # 0 capture rate = 0.979951 
    class # 1 capture rate = 0.984139 
TRAIN: Batch: 0.96943163161598 Loss: 0.0064938664
accuracy = 0.973925
mean IU  = 0.814848
    class # 0 capture rate = 0.972978 
    class # 1 capture rate = 0.991764 
TRAIN: Batch: 0.9733406301305606 Loss: 0.005890378
accuracy = 0.978613
mean IU  = 0.811831
    class # 0 capture rate = 0.978391 
    class # 1 capture rate = 0.983990 
TRAIN: Batch: 0.9772496286451411 Loss: 0.006254945
accuracy = 0.975234
mean IU  = 0.841877
    class # 0 capture rate = 0.974217 
    class # 1 capture rat

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 97.251643%. Class 0 capture: 97.237601%. Class 1 capture: 97.473441%
Character error rate improved, save model
Done with training at epoch 30 sigoptObservation=0.9725164285775644
